In [12]:
import numpy as np
import pandas as pd 
import os
import imutils
import dlib # run "pip install dlib"
import cv2 # run "pip install opencv-python"

import imageio
from imutils import face_utils

In [11]:


def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y

    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)
    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)
    return coords


In [10]:
def crop_and_save_image(img, img_path, write_img_path, img_name):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('D:/Projects/shape_predictor_68_face_landmarks/shape_predictor_68_face_landmarks/shape_predictor_68_face_landmarks.dat')
    # load the input image, resize it, and convert it to grayscale

    image = cv2.imread(img_path)
    image = imutils.resize(image, width=500)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
    rects = detector(gray, 1)
    if len(rects) > 1:
        print("Error")
        return
    if len(rects) < 1:
        print( "ERROR: no faces detected")
        return
    for (i, rect) in enumerate(rects):
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        name, i, j = 'mouth', 48, 68
        # clone = gray.copy()
        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))        
        roi = gray[y:y+h, x:x+w]
        roi = imutils.resize(roi, width = 250, inter=cv2.INTER_CUBIC)        
        # print('D:/Projects/MIRACL-VC1/cropped/' + write_img_path)
        cv2.imwrite('D:/Projects/MIRACL-VC1/cropped/' + write_img_path, roi)
        

In [14]:
people = ['F01','F02','F04','F05','F06','F07','F08','F09', 'F10','F11','M01','M02','M04','M07','M08']
data_types = ['words']
folder_enum = ['01','02','03','04','05','06','07','08', '09', '10']
instances = ['01','02','03','04','05','06','07','08', '09', '10']

words = ['Begin', 'Choose', 'Connection', 'Navigation', 'Next', 'Previous', 'Start', 'Stop', 'Hello', 'Web']          
words_di = {i:words[i] for i in range(len(words))}

In [6]:
if not os.path.exists('D:/Projects/MIRACL-VC1/cropped'):
    os.mkdir('D:/Projects/MIRACL-VC1/cropped')


![Demo](https://i.imgur.com/z6YIvFL.png)


In [15]:
import shutil

def crop_one_person():      
    if not os.path.exists('D:/Projects/MIRACL-VC1/cropped'):
        os.mkdir('D:/Projects/MIRACL-VC1/cropped')
    crp = 'D:/Projects/MIRACL-VC1/cropped/'
    # people = ['F01']
    # data_types = ['words']
    # folder_enum = ['01']
    # instances = ['01']

    # i = 1
    for person_ID in people:
        if not os.path.exists(crp + person_ID ):
            os.mkdir(crp + person_ID + '/')

        for data_type in data_types:
            if not os.path.exists(crp + person_ID + '/' + data_type):
                os.mkdir(crp + person_ID + '/' + data_type)

            for phrase_ID in folder_enum:
                if not os.path.exists(crp + person_ID + '/' + data_type + '/' + phrase_ID):
                    # F01/phrases/01
                    os.mkdir(crp + person_ID + '/' + data_type + '/' + phrase_ID)

                for instance_ID in instances:
                    # F01/phrases/01/01
                    directory = f'D:/Projects/MIRACL-VC1/dataset/{person_ID}/{data_type}/{phrase_ID}/{instance_ID}/'
                    dir_temp = person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID + '/'
    #                 print(directory)
                    filelist = os.listdir(directory)
                    if not os.path.exists(crp + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID):
                        os.mkdir(crp + person_ID + '/' + data_type + '/' + phrase_ID + '/' + instance_ID)

                        for img_name in filelist:
                            if img_name.startswith('color'):
                                image = imageio.imread(directory + '' + img_name)
                                crop_and_save_image(image, directory + '' + img_name,
                                                    dir_temp + '' + img_name, img_name)

    # print(f'Iteration : {i}')
    # i += 1
    # shutil.rmtree('D:/Projects/MIRACL-VC1/cropped/')

In [7]:
dlib.DLIB_USE_CUDA

True

In [8]:
print (dlib.cuda.get_num_devices())

1
